<a href="https://colab.research.google.com/github/Annmodels/mnist_recon/blob/master/BiGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

In [0]:
!mkdir MNIST_Fashion
!mv *.gz MNIST_Fashion/

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_Fashion/')

In [0]:
plt.figure(figsize = (1,1))
sample_image = mnist.train.next_batch(1)[0]
sample_image = sample_image.reshape([28,28])
plt.imshow(sample_image , cmap = "gray")
plt.show()

In [0]:
lrate = 0.0002
batch_size = 100
epochs = 120000

In [0]:
image_dim = 784
z_dim = 64
en_h1 = 256
d_h1 = 256
disc_h1 = 256

In [0]:
def xavier_init(shape):
  return tf.random_normal(shape = shape , stddev = 1./tf.sqrt(shape[0]/2.0))

In [0]:
en_wt = {
    "en_h1": tf.Variable(xavier_init([image_dim , en_h1])),
    "en_final": tf.Variable(xavier_init([en_h1 , z_dim]))
        }
d_wt = {
    "d_h1": tf.Variable(xavier_init([z_dim , d_h1])),
    "d_final": tf.Variable(xavier_init([d_h1 , image_dim]))
       }
disc_wt = {
    "disc_h1": tf.Variable(xavier_init([image_dim+z_dim , disc_h1])),
    "disc_final": tf.Variable(xavier_init([disc_h1 , 1])),
          }

en_bias = {
    "en_h1": tf.Variable(xavier_init([en_h1])),
    "en_final": tf.Variable(xavier_init([z_dim]))
          }
d_bias = {
    "d_h1": tf.Variable(xavier_init([d_h1])),
    "d_final": tf.Variable(xavier_init([image_dim]))
          }
disc_bias = {
    "disc_h1": tf.Variable(xavier_init([disc_h1])),
    "disc_final": tf.Variable(xavier_init([1]))
           }

In [0]:
z_input = tf.placeholder(tf.float32 , shape = [None , z_dim])
x_input = tf.placeholder(tf.float32 , shape = [None , image_dim])

In [0]:
def z_distribution(m,n):
  return np.random.uniform(-1.,1.,size = [m,n])
  

In [0]:
def Discriminator(x,z):
  input = tf.concat(axis = 1 , values = [x,z])
  disc_hidn_layer = tf.nn.relu(tf.add(tf.matmul(input , disc_wt["disc_h1"]) , disc_bias["disc_h1"]))
  final_layer = tf.add(tf.matmul(disc_hidn_layer , disc_wt["disc_final"]) , disc_bias["disc_final"])
  disc_prob_output = tf.nn.sigmoid(final_layer)
  return disc_prob_output

def Encoder(x):
  en_hidn_layer = tf.nn.relu(tf.add(tf.matmul(x , en_wt["en_h1"]) , en_bias["en_h1"]))
  final_layer = tf.add(tf.matmul(en_hidn_layer , en_wt["en_final"]) , en_bias["en_final"])
  en_output = final_layer
  return en_output

def Decoder(z):
  d_hidn_layer = tf.nn.relu(tf.add(tf.matmul(z , d_wt["d_h1"]) , d_bias["d_h1"]))
  final_layer = tf.add(tf.matmul(d_hidn_layer , d_wt["d_final"]) , d_bias["d_final"])
  gen_prob_output = tf.nn.sigmoid(final_layer)
  return gen_prob_output

In [0]:
z_generated = Encoder(x_input)
x_generated = Decoder(z_input)
real_output_disc = Discriminator(x_input,z_generated)
fake_output_disc = Discriminator(x_generated,z_input)

In [0]:
Discriminator_Loss = -tf.reduce_mean(tf.log(real_output_disc + 1e-7)+tf.log(1.0-fake_output_disc + 1e-7))
Decoder_Loss = -tf.reduce_mean(tf.log(fake_output_disc + 1e-7)+tf.log(1.0-real_output_disc + 1e-7))

In [0]:
Encoder_var = [en_wt["en_h1"],en_wt["en_final"],en_bias["en_h1"],en_bias["en_final"]]
Decoder_var = [d_wt["d_h1"],d_wt["d_final"],d_bias["d_h1"],d_bias["d_final"]]
Discriminator_var = [disc_wt["disc_h1"],disc_wt["disc_final"],disc_bias["disc_h1"],disc_bias["disc_final"]]

In [0]:
with tf.name_scope("Optimizer_Discriminator")as scope:
  Discriminator_optimize = tf.train.AdamOptimizer(learning_rate = lrate).minimize(Discriminator_Loss , var_list = Discriminator_var)

with tf.name_scope("Optimizer_Decoder")as scope:
  Decoder_optimize = tf.train.AdamOptimizer(learning_rate = lrate).minimize(Decoder_Loss , var_list = Encoder_var+Decoder_var)

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  x_batch,_ = mnist.train.next_batch(batch_size)
  z_noise = z_distribution(batch_size,z_dim)
  #np.random.uniform(-1.,1.,size = [batch_size , z_noise_dim]) 
  _,Disc_loss_epoch = sess.run([Discriminator_optimize,Discriminator_Loss],feed_dict = {x_input:x_batch,z_input:z_noise}) 
  _,D_loss_epoch = sess.run([Decoder_optimize,Decoder_Loss],feed_dict = {x_input:x_batch,z_input:z_noise})
  if epoch%2000 == 0:
    print("steps: {0}  D_loss: {1} , Disc_loss: {2}".format(epoch,D_loss_epoch,Disc_loss_epoch))

In [0]:
test_output = Decoder(z_input)
n = 6
canvas = np.empty((28*n,28*n))
for i in range(n):
  z_noise = z_distribution(batch_size,z_dim)
  #np.random.uniform(-1.,1.,size = [batch_size , z_noise_dim])
  g = sess.run(test_output , feed_dict = {z_input:z_noise})
  g = -1*(g-1)
  for j in range(n):
    canvas[i*28:(i+1)*28 , j*28:(j+1)*28] = g[j].reshape([28,28])

plt.figure(figsize = (n,n))
plt.imshow(canvas , origin = "upper" , cmap = "gray")
plt.show()

In [0]:
z_generated = Encoder(x_input)
test_output = Decoder(z_generated)
n = 6
canvas1 = np.empty((28*n , 28*n))
canvas2 = np.empty((28*n , 28*n))

for i in range(n):
  x_batch,_ = mnist.train.next_batch(batch_size)
  g_x = sess.run(test_output,feed_dict = {x_input:x_batch})
  #print(g.shape)
  for j in range(n):
    canvas1[i*28:(i+1)*28,j*28:(j+1)*28] = g_x[j].reshape([28,28])
  for j in range(n):
    canvas2[i*28:(i+1)*28,j*28:(j+1)*28] = x_batch[j].reshape([28,28])

f,ax = plt.subplots(1,2)
ax[0].imshow(canvas1 , origin = "upper" , cmap = "gray")
ax[1].imshow(canvas2 , origin = "upper" , cmap = "gray")
plt.show()


    